In [1]:
#2020Sep22. Ting Sun
#GSE116470
#mouse healthy samples
#snRNA-seq
#matix organization and seurat object forming

#under sc_env

In [1]:
#for scRNA-seq objects
library(Seurat)
library(dplyr)
library(Matrix)
library(abind)

#for visualization
library(cowplot)
library(ggplot2)

#for DGE analysis
library(DESeq2)
library(MAST)
library(patchwork)
library(pheatmap)


#normally unnecessary, for loading sparse matrix
library(DropSeq.util)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


********************************************************
Note: As of version 1.0.0, cowplot does not change the
  default ggplot2 theme anymore. To recover the previous
  behavior, execute:
  theme_set(theme_cowplot())
********************************************************

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following object is masked from ‘package:Matrix’:

    which

The following objects are masked from ‘packa

ERROR: Error in library(pheatmap): there is no package called ‘pheatmap’


In [3]:
data_files<-list.files(path = "/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/raw",
                    pattern = "*.txt")

In [4]:
data_files

[1] "GSE116470_F_GRCm38.81.P60Cerebellum_ALT.raw.dge.txt"             
[2] "GSE116470_F_GRCm38.81.P60Cortex_noRep5_FRONTALonly.raw.dge.txt"  
[3] "GSE116470_F_GRCm38.81.P60Cortex_noRep5_POSTERIORonly.raw.dge.txt"
[4] "GSE116470_F_GRCm38.81.P60EntoPeduncular.raw.dge.txt"             
[5] "GSE116470_F_GRCm38.81.P60GlobusPallidus.raw.dge.txt"             
[6] "GSE116470_F_GRCm38.81.P60Hippocampus.raw.dge.txt"                
[7] "GSE116470_F_GRCm38.81.P60Striatum.raw.dge.txt"                   
[8] "GSE116470_F_GRCm38.81.P60SubstantiaNigra.raw.dge.txt"            
[9] "GSE116470_F_GRCm38.81.P60Thalamus.raw.dge.txt"

In [5]:
setwd("/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/raw")

In [6]:
#according to the original paper's protocol
#first instal customized package for installing matrix
#reference from http://dropviz.org/

#install.packages("/home/tsun/Downloads/DropSeq.util_2.0.tar.gz", repos=NULL)

Warning message in install.packages("/home/tsun/Downloads/DropSeq.util_2.0.tar.gz", :
“installation of package ‘/home/tsun/Downloads/DropSeq.util_2.0.tar.gz’ had non-zero exit status”

In [7]:
#library(DropSeq.util)

In [8]:
dge.path2<-"/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/raw/GSE116470_F_GRCm38.81.P60Thalamus.raw.dge.txt"
dge2<-loadSparseDge(dge.path2)

In [9]:
str(dge2)

Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:96610803] 0 1 1 1 1 1 1 1 1 1 ...
  ..@ j       : int [1:96610803] 684 1 5 6 11 13 14 15 27 30 ...
  ..@ Dim     : int [1:2] 27385 127375
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:27385] "0610005C13Rik" "0610007P14Rik" "0610009B22Rik" "0610009E02Rik" ...
  .. ..$ : chr [1:127375] "P60THRep1P1_GTGTCACGATTT" "P60THRep1P1_GGAGTTAGGTGT" "P60THRep1P1_ATTCACAATCCC" "P60THRep1P1_CACAATAAATTT" ...
  ..@ x       : num [1:96610803] 1 2 1 1 1 1 3 1 3 1 ...
  ..@ factors : list()


In [10]:
dim(dge2)

[1]  27385 127375

In [11]:
class(dge2)

[1] "dgTMatrix"
attr(,"package")
[1] "Matrix"

In [12]:
dge2[1:6,1:6]

6 x 6 sparse Matrix of class "dgTMatrix"
              P60THRep1P1_GTGTCACGATTT P60THRep1P1_GGAGTTAGGTGT
0610005C13Rik                        .                        .
0610007P14Rik                        .                        2
0610009B22Rik                        .                        .
0610009E02Rik                        .                        .
0610009L18Rik                        .                        .
0610009O20Rik                        .                        .
              P60THRep1P1_ATTCACAATCCC P60THRep1P1_CACAATAAATTT
0610005C13Rik                        .                        .
0610007P14Rik                        .                        .
0610009B22Rik                        .                        .
0610009E02Rik                        .                        .
0610009L18Rik                        .                        .
0610009O20Rik                        .                        1
              P60THRep1P1_GCTTGTAGGAGA P60THRep1P1_GTCTCACTTAGT

In [13]:
test2<-CreateSeuratObject(dge2, assay = "RNA", project = "test")

In [14]:
test2

An object of class Seurat 
27385 features across 127375 samples within 1 assay 
Active assay: RNA (27385 features)

In [28]:
head(test2@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA
P60THRep1P1_GTGTCACGATTT,P60THRep1P1,24716,6549
P60THRep1P1_GGAGTTAGGTGT,P60THRep1P1,21620,5263
P60THRep1P1_ATTCACAATCCC,P60THRep1P1,17511,4352
P60THRep1P1_CACAATAAATTT,P60THRep1P1,17007,4241
P60THRep1P1_GCTTGTAGGAGA,P60THRep1P1,15295,4568
P60THRep1P1_GTCTCACTTAGT,P60THRep1P1,14451,4245


In [16]:
ting_read<-function(x){
    dge.path<-paste0("/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/raw/",
                     x)
    dge<-loadSparseDge(dge.path)
    return(dge)
                     }

In [17]:
data_list<-lapply(X = data_files, FUN = ting_read)

In [18]:
str(data_list)

List of 9
 $ :Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  .. ..@ i       : int [1:29200479] 0 0 0 0 0 0 0 1 1 1 ...
  .. ..@ j       : int [1:29200479] 33 407 856 3526 5572 5619 7349 1 4 5 ...
  .. ..@ Dim     : int [1:2] 24247 77147
  .. ..@ Dimnames:List of 2
  .. .. ..$ : chr [1:24247] "0610005C13Rik" "0610007P14Rik" "0610009B22Rik" "0610009E02Rik" ...
  .. .. ..$ : chr [1:77147] "P60CBRep1P1_GACTCTACACCC" "P60CBRep1P1_TATTATATCTAA" "P60CBRep1P1_GCGTCGCCAGTT" "P60CBRep1P1_CTATCGATTTCN" ...
  .. ..@ x       : num [1:29200479] 2 1 1 1 1 1 2 1 1 1 ...
  .. ..@ factors : list()
 $ :Formal class 'dgTMatrix' [package "Matrix"] with 6 slots
  .. ..@ i       : int [1:222528087] 0 0 0 0 0 0 0 0 0 0 ...
  .. ..@ j       : int [1:222528087] 443 1135 2757 5319 6061 7486 7569 8116 8404 9095 ...
  .. ..@ Dim     : int [1:2] 29463 194027
  .. ..@ Dimnames:List of 2
  .. .. ..$ : chr [1:29463] "0610005C13Rik" "0610007P14Rik" "0610009B22Rik" "0610009E02Rik" ...
  .. .. ..$ : chr [1:19

In [19]:
data_files

[1] "GSE116470_F_GRCm38.81.P60Cerebellum_ALT.raw.dge.txt"             
[2] "GSE116470_F_GRCm38.81.P60Cortex_noRep5_FRONTALonly.raw.dge.txt"  
[3] "GSE116470_F_GRCm38.81.P60Cortex_noRep5_POSTERIORonly.raw.dge.txt"
[4] "GSE116470_F_GRCm38.81.P60EntoPeduncular.raw.dge.txt"             
[5] "GSE116470_F_GRCm38.81.P60GlobusPallidus.raw.dge.txt"             
[6] "GSE116470_F_GRCm38.81.P60Hippocampus.raw.dge.txt"                
[7] "GSE116470_F_GRCm38.81.P60Striatum.raw.dge.txt"                   
[8] "GSE116470_F_GRCm38.81.P60SubstantiaNigra.raw.dge.txt"            
[9] "GSE116470_F_GRCm38.81.P60Thalamus.raw.dge.txt"

In [20]:
brain_region_list<-c("Cerebellum","FrontalCortex","PosteriorCortex",
                    "EntoPeduncular","GlobusPallidus",
                    "Hippocampus","Striatum","SubstantiaNigra",
                    "Thalamus")

In [21]:
#create list of seurat objects

seurat_list<-vector(mode = "list", length = 9)

for(i in 1:9){
    seurat_list[[i]]<-CreateSeuratObject(counts = data_list[[i]], 
                                         project = 'Saunders2018', 
                                         assay = 'RNA')
    seurat_list[[i]]@meta.data$BrainRegion<-brain_region_list[[i]]
    seurat_list[[i]]@meta.data$Study<-"Saunders_2018"
    seurat_list[[i]]@meta.data$CellTypeOriginal<-NA
    seurat_list[[i]]@meta.data$CellTypeSubcluster<-NA
    seurat_list[[i]]@meta.data$Age<-"P60"
}

In [22]:
seurat_list
head(seurat_list[[1]]@meta.data)

[[1]]
An object of class Seurat 
24247 features across 77147 samples within 1 assay 
Active assay: RNA (24247 features)

[[2]]
An object of class Seurat 
29463 features across 194027 samples within 1 assay 
Active assay: RNA (29463 features)

[[3]]
An object of class Seurat 
27720 features across 116437 samples within 1 assay 
Active assay: RNA (27720 features)

[[4]]
An object of class Seurat 
22631 features across 33761 samples within 1 assay 
Active assay: RNA (22631 features)

[[5]]
An object of class Seurat 
27091 features across 90300 samples within 1 assay 
Active assay: RNA (27091 features)

[[6]]
An object of class Seurat 
27953 features across 134430 samples within 1 assay 
Active assay: RNA (27953 features)

[[7]]
An object of class Seurat 
25645 features across 90628 samples within 1 assay 
Active assay: RNA (25645 features)

[[8]]
An object of class Seurat 
26434 features across 75384 samples within 1 assay 
Active assay: RNA (26434 features)

[[9]]
An object of class Seur

,orig.ident,nCount_RNA,nFeature_RNA,BrainRegion,Study,CellTypeOriginal,CellTypeSubcluster,Age
P60CBRep1P1_GACTCTACACCC,P60CBRep1P1,11903,3873,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_TATTATATCTAA,P60CBRep1P1,11446,3587,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_GCGTCGCCAGTT,P60CBRep1P1,9617,3501,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_CTATCGATTTCN,P60CBRep1P1,9609,3414,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_GTACCTGAGCCT,P60CBRep1P1,8849,3637,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_TCGCTTAGGCAT,P60CBRep1P1,8088,2508,Cerebellum,Saunders_2018,NA,NA,P60


In [23]:
meta_list<-list.files(path = "/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/meta/cluster_rds/",
                    pattern = "*.RDS")

In [54]:
meta_list

[1] "H_1stRound_CrossTissue_Astrocytes_9-13-17.cell_cluster_outcomes.RDS"         
[2] "H_1stRound_CrossTissue_Endothelial_5-3-17.cell_cluster_outcomes.RDS"         
[3] "H_1stRound_CrossTissue_FibroblastLike_5-3-17.cell_cluster_outcomes.RDS"      
[4] "H_1stRound_CrossTissue_Microglia_Macrophage_5-3-17.cell_cluster_outcomes.RDS"
[5] "H_1stRound_CrossTissue_Mural_5-3-17.cell_cluster_outcomes.RDS"               
[6] "H_1stRound_CrossTissue_Oligodendrocytes_5-3-17.cell_cluster_outcomes.RDS"    
[7] "H_1stRound_CrossTissue_Polydendrocytes_5-3-17.cell_cluster_outcomes.RDS"

In [55]:
setwd("/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/meta/cluster_rds")
meta<-lapply(meta_list,readRDS)

In [56]:
head(meta[[6]])
unique(meta[[1]]$reason)
dim(meta[[5]])

,cluster,subcluster,reason
P60HippoRep1P1_GCACCGCCCGAA,1,1-3,NA
P60HippoRep1P1_TCTATGATCAAC,1,1-3,NA
P60HippoRep1P1_CAATTCGTGCCC,1,1-2,NA
P60HippoRep1P1_GGTTCGTCAGGA,1,1-2,NA
P60HippoRep1P1_TTCCACCACAAA,1,1-2,NA
P60HippoRep1P1_CTGATCGAGAAA,1,1-3,NA


[1] <NA>                 doublet              outlier             
[4] singleton_subcluster
Levels: doublet outlier singleton_subcluster

[1] 5357    3

In [57]:
cell_type_list<-c("Astrocyte","Endothelial","FibroblastLike","Microglia",
                 "Mural","Oligodendrocytes","Polydendrocytes")

In [58]:
for(i in 1:7){
    meta[[i]]$cluster<-cell_type_list[[i]]
}

In [59]:
head(meta[[1]])

,cluster,subcluster,reason
P60HippoRep1P1_TAGTCAAGCAAC,Astrocyte,1-7,NA
P60HippoRep1P1_GGCCTCCCCTCT,Astrocyte,1-7,NA
P60HippoRep1P1_ATTTCGCCAGCC,Astrocyte,1-7,NA
P60HippoRep1P1_CAGTCAACTCCT,Astrocyte,1-7,NA
P60HippoRep1P1_CCATGGTTTACN,Astrocyte,1-7,NA
P60HippoRep1P1_TCCCGCGACGTA,Astrocyte,1-7,NA


In [60]:
meta.test<-rbind(meta[[1]],meta[[2]])
head(meta.test)
tail(meta.test)

,cluster,subcluster,reason
P60HippoRep1P1_TAGTCAAGCAAC,Astrocyte,1-7,NA
P60HippoRep1P1_GGCCTCCCCTCT,Astrocyte,1-7,NA
P60HippoRep1P1_ATTTCGCCAGCC,Astrocyte,1-7,NA
P60HippoRep1P1_CAGTCAACTCCT,Astrocyte,1-7,NA
P60HippoRep1P1_CCATGGTTTACN,Astrocyte,1-7,NA
P60HippoRep1P1_TCCCGCGACGTA,Astrocyte,1-7,NA


,cluster,subcluster,reason
P60CBRep3P2_CGGAATCGCCAN,Endothelial,1-2,NA
P60CBRep3P2_GCGGTATGGATT,Endothelial,1,doublet
P60CBRep3P2_TACCTTGACTAA,Endothelial,1-5,NA
P60CBRep3P2_TATGGACTGTCA,Endothelial,1-2,NA
P60CBRep3P2_CAATCTAACAGC,Endothelial,1-5,NA
P60CBRep3P2_TCCACTTATTGC,Endothelial,1-2,NA


In [61]:
meta_all<-Reduce(function(x,y) rbind(x,y), meta)

In [62]:
dim(meta_all)

[1] 109799      3

In [63]:
head(meta_all)

,cluster,subcluster,reason
P60HippoRep1P1_TAGTCAAGCAAC,Astrocyte,1-7,NA
P60HippoRep1P1_GGCCTCCCCTCT,Astrocyte,1-7,NA
P60HippoRep1P1_ATTTCGCCAGCC,Astrocyte,1-7,NA
P60HippoRep1P1_CAGTCAACTCCT,Astrocyte,1-7,NA
P60HippoRep1P1_CCATGGTTTACN,Astrocyte,1-7,NA
P60HippoRep1P1_TCCCGCGACGTA,Astrocyte,1-7,NA


In [64]:
unique(meta_all$cluster)

[1] "Astrocyte"        "Endothelial"      "FibroblastLike"   "Microglia"       
[5] "Mural"            "Oligodendrocytes" "Polydendrocytes"

In [65]:
#prepare merged seurat object
seu<-merge(seurat_list[[1]],y = c(seurat_list[c(2,3,6,7)]), project = "Saunders_2018")

In [66]:
seu

An object of class Seurat 
31487 features across 612669 samples within 1 assay 
Active assay: RNA (31487 features)

In [67]:
head(seu@meta.data)

,orig.ident,nCount_RNA,nFeature_RNA,BrainRegion,Study,CellTypeOriginal,CellTypeSubcluster,Age
P60CBRep1P1_GACTCTACACCC,P60CBRep1P1,11903,3873,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_TATTATATCTAA,P60CBRep1P1,11446,3587,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_GCGTCGCCAGTT,P60CBRep1P1,9617,3501,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_CTATCGATTTCN,P60CBRep1P1,9609,3414,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_GTACCTGAGCCT,P60CBRep1P1,8849,3637,Cerebellum,Saunders_2018,NA,NA,P60
P60CBRep1P1_TCGCTTAGGCAT,P60CBRep1P1,8088,2508,Cerebellum,Saunders_2018,NA,NA,P60


In [68]:
#intermediate saving step
saveRDS(seu,"/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/rds/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA_5regions_merged.rds")

In [71]:
for(k in 1:9){
    saveRDS(seurat_list[k], file = paste0("/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/rds/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA_",
                            brain_region_list[[k]],
                                           "_raw.rds"))
}

In [72]:
#in merged dataset
#stack in cell population information and run until UMAP embedding
#to cross validate
for(i in 1:nrow(seu@meta.data)){
    position<-match(rownames(seu@meta.data)[[i]],
                    rownames(meta_all))
    seu@meta.data$CellTypeOriginal[[i]]<-meta_all$cluster[position]
}

In [73]:
seu@meta.data.data$CellTypeOriginal[is.na(seu@meta.data$CellTypeOriginal)]<-"Other"

In [74]:
head(seu@meta.data)
unique(seu@meta.data$CellTypeOriginal)

,orig.ident,nCount_RNA,nFeature_RNA,BrainRegion,Study,CellTypeOriginal,CellTypeSubcluster,Age
P60CBRep1P1_GACTCTACACCC,P60CBRep1P1,11903,3873,Cerebellum,Saunders_2018,Other,NA,P60
P60CBRep1P1_TATTATATCTAA,P60CBRep1P1,11446,3587,Cerebellum,Saunders_2018,Other,NA,P60
P60CBRep1P1_GCGTCGCCAGTT,P60CBRep1P1,9617,3501,Cerebellum,Saunders_2018,Endothelial,NA,P60
P60CBRep1P1_CTATCGATTTCN,P60CBRep1P1,9609,3414,Cerebellum,Saunders_2018,Other,NA,P60
P60CBRep1P1_GTACCTGAGCCT,P60CBRep1P1,8849,3637,Cerebellum,Saunders_2018,Other,NA,P60
P60CBRep1P1_TCGCTTAGGCAT,P60CBRep1P1,8088,2508,Cerebellum,Saunders_2018,Oligodendrocytes,NA,P60


[1] "Other"            "Endothelial"      "Oligodendrocytes" "Astrocyte"       
[5] "Mural"            "Microglia"        "Polydendrocytes"  "FibroblastLike"

In [76]:
seu@meta.data$CellType<-seu@meta.data$CellTypeOriginal

In [77]:
seu@meta.data$CellType<-gsub("Oligodendrocytes","MOL",
                            seu@meta.data$CellType)

In [78]:
unique(seu@meta.data$CellType)

[1] "Other"           "Endothelial"     "MOL"             "Astrocyte"      
[5] "Mural"           "Microglia"       "Polydendrocytes" "FibroblastLike"

In [79]:
saveRDS(seu,"/media/tsun/Data/Tsun/external_data/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA/rds/GSE116470_Mouse_Saunders_McCarroll_Cell_2018_scRNA_5regions_merged.rds")